In [43]:
import matplotlib.pyplot as plt
import scipy.linalg as linalg
import numpy as np
import pandas 
import math
#import geopandas

In [55]:
x_Geodetic_0 = np.array([44.39, 8.93889, 70])
#x_Geodetic_0 = np.array([[44, 23, 24], [8, 56, 20], 70])
sigma_0 = 10
x_LL_A = np.array([0,30, 0])
x_LL_B = np.array([0, -30, 0])
x_LL_C = np.array([200, 0, 0])
sigma_A = 2
sigma_B = 2
sigma_C = 10

csi = np.array([0, 0, 10.23])
eta = np.array([0, 0, 9.5])
alpha = np.array([30, 27, 18])

lambd = x_Geodetic_0[1]
phi = x_Geodetic_0[0]
h = x_Geodetic_0[2]

a = 6378137
e2 = 0.0818191908426215**2
b = np.sqrt((a**2-1)/e2)

R_N = a/np.sqrt(1 - e2*np.power(np.sin(phi),2))

In [56]:
def sex2deg(a):
    return(a[0] + a[1]/60 + a[2]/3600)

In [57]:
def deg2rad(a):
    return(a*np.pi/180)

In [72]:
def deg2sex(a):
    deg = np.fix(a)
    mins = (a - deg)
    secs = (mins - np.fix(mins))*60
    return([deg, mins, secs])

In [42]:
def rad2deg(a):
    return(a*180/np.pi)

In [58]:
def R():
    a1 = deg2rad(sex2deg(csi))
    a2 = deg2rad(sex2deg(eta))
    a3 = deg2rad(sex2deg(alpha))
    R_x = np.array([[1, 0, 0],
                   [0, np.cos(a1), -np.sin(a1)],
                   [0, np.sin(a1), np.cos(a1)]])
    R_y = np.array([[np.cos(a2), 0, -np.sin(a2)],
                   [0, 1, 0],
                   [np.sin(a2), 0, np.cos(a2)]])
    R_z = np.array([[np.cos(a3), np.sin(a3), 0],
                   [-np.sin(a3), np.cos(a3), 0], 
                   [0, 0, 1]])
    return(np.dot(R_z,R_y,R_x))

In [59]:
def R_LL2LC(x_LL):
    
    return(np.dot(np.transpose(R()),x_LL))

In [60]:
def R_0(l,fi):
    return(np.array([[-np.sin(l), np.cos(l), 0],
         [-np.sin(fi)*np.cos(l), -np.sin(fi)*np.sin(l), np.cos(fi)],
         [ np.cos(fi)*np.cos(l), np.cos(fi)*np.sin(l), np.sin(fi)]]))

In [61]:
def R_LC2GC(x_LC,x_GC_0):
    return(x_GC_0+np.dot(np.transpose(R_0(deg2rad(lambd),deg2rad(phi))),x_LC))

In [67]:
def Geodetic2GC():
    
    x = (R_N + h)*np.cos(deg2rad(phi))*np.cos(deg2rad(lambd))
    y = (R_N + h)*np.cos(deg2rad(phi))*np.sin(deg2rad(lambd))
    z = (R_N*(1-e2) + h)*np.sin(deg2rad(phi))

    return(np.array([x, y, z]))

In [62]:
def covariance(x_GC_0,X,Y,Z):
    r = np.sqrt(X**2,Y**2)
    Fi = math.atan2(Z,r*np.sqrt(1-e2))
    l = math.atan2(Y,X)
    fi = math.atan2(Z+e2*b*np.sin(Fi)**3,r-e2*a*cos(Fi)**3)


    C_LL = np.array([[sigma_A**2,0,0],
                     [0,sigma_B**2,0],
                     [0,0,sigma_C**2]])

    C_LC = np.dot(R(),C_LL,np.transpose(R()))
    
    C_GC = x_GC_0 + np.dot(np.transpose(R_0()),C_LC,R_0())

    C_fin = np.dot(R_0(l,fi),C_GC,np.transpose(R_0(l,fi)))
    
    return 

In [61]:
A= np.array([[1,0,0],[0,0,1],[0,1,0]])
B= np.array([[1,0,1],[2,0,1],[0,1,0]])
C= np.array([[0],[1],[0]])
D= np.array([1,0,0])
print(np.dot(A,C)+D)

[[1 0 0]
 [1 0 0]
 [2 1 1]]


In [71]:
np.set_printoptions(precision=3)
x_GC_0 = Geodetic2GC()
print('x_GC_0 =',x_GC_0)
x_LC_A = R_LL2LC(x_LL_A)
x_LC_B = R_LL2LC(x_LL_B)
x_LC_C = R_LL2LC(x_LL_C)
print('x_LC_A =',x_LC_A)
print('x_LC_B =',x_LC_B)
print('x_LC_C =',x_LC_C)

x_GC_A = R_LC2GC(x_LC_A,x_GC_0)
x_GC_B = R_LC2GC(x_LC_B,x_GC_0)
x_GC_C = R_LC2GC(x_LC_C,x_GC_0)
print('x_GC_A =',x_GC_A)
print('x_GC_B =',x_GC_B)
print('x_GC_C =',x_GC_C)

# from EPN website
x_GC_A_ETRF = [4504832.9723   708554.9441  4434285.6299 ]
x_GC_B_ETRF = [4504863.523    708591.225   4434249.043  ]
x_GC_C_ETRF = [4504751.6333   708732.0681  4434339.5839 ]
# from ETRF to Geodetic (remeber: lat and lon in sex)

x_GC_0 = [4504848.015  708573.393 4434267.524]
x_LC_A = [-1.521e+01  2.586e+01  7.003e-04]
x_LC_B = [ 1.521e+01 -2.586e+01 -7.003e-04]
x_LC_C = [ 1.724e+02  1.014e+02 -7.941e-03]
x_GC_A = [4504832.507  708555.561 4434286.004]
x_GC_B = [4504863.523  708591.225 4434249.043]
x_GC_C = [4504751.168  708732.685 4434339.958]
